In [1]:
import os
import csv
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../input_data_clean/bank_tract_clean_WITH_CENSUS.csv')

In [3]:
hmda_vars = ['num_approved_loans', 
             'num_applications',
             'sum_approved_loans',
             'sum_all_loan_amount',
             'denial_rate',
             'avg_approved_loan_size'
             #'median__all_income',
             #'sum_rate_spread',
             #'mean_LTV'
            ]

In [4]:
data['pct_minority'] = (data['Tot.Pop'] - data['Tot.WhitePop'])/data['Tot.Pop'] * 100
#data['pct_minority'] = data['pct_minority'].apply(lambda x: "{:.2f%}".format(x))

In [5]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='pct_minority', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['pct_minority_Quintiles'] = pd.qcut(data['pct_minority'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['pct_minority_Quintiles'] = pd.cut(data['pct_minority'], bins=5, labels=['20', '40', '60', '80', '100'])

In [6]:
pd.set_option('display.max_rows', 100)

In [7]:
Quintiles_loan_table = data.groupby('which_bank')[hmda_vars].mean()
#Quintiles_loan_table['loan_approvel_rate'] = Quintiles_loan_table['num_approved_loans']/Quintiles_loan_table['num_applications']
#Quintiles_loan_table = Quintiles_loan_table.sort_values(by='loan_approvel_rate', ascending=False)
Quintiles_loan_table

,num_approved_loans,num_applications,sum_approved_loans,sum_all_loan_amount,denial_rate,avg_approved_loan_size
which_bank,,,,,,
All Other Banks,54.401735,64.671093,2.576171e+07,2.963290e+07,0.831372,483392.887220
Bank of West,2.394889,3.379333,1.312934e+06,1.767377e+06,0.795004,480744.003676


In [8]:
Quintiles_loan_table3 = data.groupby(['which_bank', "pct_minority_Quintiles"])[hmda_vars].sum()
Quintiles_loan_table3['loan_pct_in_minority_group'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['$loan_pct_in_minority_group'] = Quintiles_loan_table3['sum_approved_loans']/Quintiles_loan_table3['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table3['loan_approval_rate'] = Quintiles_loan_table3['num_approved_loans']/Quintiles_loan_table3['num_applications']
Quintiles_loan_table3.iloc[:,-3:]

loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                               
All Other Banks 20                                        0.318246   
                40                                        0.365025   
                60                                        0.189934   
                80                                        0.101473   
                100                                       0.025322   
Bank of West    20                                        0.290897   
                40                                        0.351304   
                60                                        0.184096   
                80                                        0.136030   
                100                                       0.037673   

                                        $loan_pct_in_minority_group  \
which_bank      pct_minority_Quintiles                                
All Other Banks 20                                         0.321018   
                40                                         0.357555   
                60                                         0.189634   
                80                                         0.103468   
                100                                        0.028325   
Bank of West    20                                         0.326797   
                40                                         0.349883   
                60                                         0.161200   
                80                                         0.122943   
                100                                        0.039178   

                                        loan_approval_rate  
which_bank      pct_minority_Quintiles                      
All Other Banks 20                                0.846332  
                40                                0.840129  
                60                                0.835710  
                80                                0.835489  
                100                               0.857581  
Bank of West    20                                0.707356  
                40                                0.718679  
                60                                0.709331  
                80                                0.688263  
                100                               0.700000

In [9]:
data['pct_White'] = data['Tot.WhitePop']/data['Tot.Pop'] * 100
data['pct_Black'] = data['Tot.BlackPop ']/data['Tot.Pop'] * 100
data['pct_AmericIndian'] = data['Tot.AmericIndianPop']/data['Tot.Pop'] * 100
data['pct_Asian'] = data['Tot.AsianPop']/data['Tot.Pop'] * 100
data['pct_NativeHawaiianPacific'] = data['Tot.NativeHawaiianPacificPop']/data['Tot.Pop'] * 100
data['pct_OtherRaceAlone'] = data['Tot.OtherRaceAlonePop']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace'] = data['Tot.TwoOrMoreRace']/data['Tot.Pop'] * 100
data['pct_TwoOrMoreRace(Some other race)'] = data['Tot.TwoOrMoreRace(Some other race)']/data['Tot.Pop'] * 100
data['pct_Hispanic/Latino'] = data['Tot.Hispanic/Latino']/data['Tot.Pop'] * 100
data['pct_NotHispanic/Latino'] = data['Tot.NotHispanic/Latino']/data['Tot.Pop'] * 100

In [10]:
data['major_minority'] = data[['pct_Black', 'pct_AmericIndian', 'pct_Asian','pct_NativeHawaiianPacific',
                                        'pct_OtherRaceAlone','pct_TwoOrMoreRace','pct_TwoOrMoreRace(Some other race)',
                                        'pct_Hispanic/Latino']].idxmax(axis=1)
data['major_minority'] = data['major_minority'].apply(lambda x: 'Black' if x=='pct_Black' 
                                                                        else 'American Indian' if x=='pct_AmericIndian' 
                                                                        else 'Asian' if x=='pct_Asian' 
                                                                        else 'Native Hawaiian Pacific' if x=='pct_NativeHawaiianPacific' 
                                                                        else 'Other Race Alone' if x=='pct_OtherRaceAlone' 
                                                                        else 'Two Or More Race' if x=='pct_TwoOrMoreRace' 
                                                                        else 'Two Or More Race(Some other race)' if x=='pct_TwoOrMoreRace(Some other race)' 
                                                                        else 'Hispanic/Latino')

In [11]:
Quintiles_loan_table4 = data.groupby(['which_bank', "major_minority"])[hmda_vars].sum()
Quintiles_loan_table4['loan_pct_in_minority_group'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['$loan_pct_in_minority_group'] = Quintiles_loan_table4['sum_approved_loans']/Quintiles_loan_table4['sum_approved_loans'].groupby("which_bank").sum()
Quintiles_loan_table4['loan_approval_rate'] = Quintiles_loan_table4['num_approved_loans']/Quintiles_loan_table4['num_applications']
Quintiles_loan_table4.iloc[:,-3:]

loan_pct_in_minority_group  \
which_bank      major_minority                                        
All Other Banks American Indian                            0.001982   
                Asian                                      0.234425   
                Black                                      0.016346   
                Hispanic/Latino                            0.736491   
                Native Hawaiian Pacific                    0.000018   
                Other Race Alone                           0.000228   
                Two Or More Race                           0.010509   
Bank of West    American Indian                            0.002691   
                Asian                                      0.364851   
                Black                                      0.017909   
                Hispanic/Latino                            0.594507   
                Other Race Alone                           0.001113   
                Two Or More Race                           0.018929   

                                         $loan_pct_in_minority_group  \
which_bank      major_minority                                         
All Other Banks American Indian                             0.001185   
                Asian                                       0.331705   
                Black                                       0.016313   
                Hispanic/Latino                             0.629610   
                Native Hawaiian Pacific                     0.000007   
                Other Race Alone                            0.000431   
                Two Or More Race                            0.020750   
Bank of West    American Indian                             0.001646   
                Asian                                       0.477665   
                Black                                       0.015276   
                Hispanic/Latino                             0.470934   
                Other Race Alone                            0.001082   
                Two Or More Race                            0.033398   

                                         loan_approval_rate  
which_bank      major_minority                               
All Other Banks American Indian                    0.813409  
                Asian                              0.880447  
                Black                              0.820617  
                Hispanic/Latino                    0.829740  
                Native Hawaiian Pacific            0.900000  
                Other Race Alone                   0.800000  
                Two Or More Race                   0.859028  
Bank of West    American Indian                    0.630435  
                Asian                              0.698898  
                Black                              0.742308  
                Hispanic/Latino                    0.714349  
                Other Race Alone                   0.631579  
                Two Or More Race                   0.710801

In [12]:
# 18.9% is the national average for hispanic/latino population
# Here, we consider anything over the national average of 18.9% a high hispanic population
# Hispanic pop in AZ = 32.3%, in CA = 39.4%
data['high_hisp_pop'] = ((data['Tot.Hispanic/Latino']/data['Tot.Pop']) > 0.189)
high_hispanic= data[data['high_hisp_pop']]

In [13]:
# finding the median income quintiles
# sort the median__all_income variable in ascending order
data.sort_values(by='median_all_income', inplace=True)

# divide the sorted median__all_income variable into 5 equal parts, or quintiles
data['Median_Income_Quintiles'] = pd.qcut(data['median_all_income'], q=5, labels=False)

# create a new variable called "Median_Income_Quintiles" based on the quintiles
data['Median_Income_Quintiles'] = pd.cut(data['Median_Income_Quintiles'], bins=5, labels=['Q1', 'Q2', 'Q3', 'Q4', 'Q5'])

In [16]:
Quintiles_loan_table = data.groupby('Median_Income_Quintiles')[hmda_vars].mean()
Quintiles_loan_table['loan_approvel_rate'] = Quintiles_loan_table['num_approved_loans']/Quintiles_loan_table['num_applications']
Quintiles_loan_table = Quintiles_loan_table.sort_values(by='loan_approvel_rate', ascending=False)
Quintiles_loan_table

,num_approved_loans,num_applications,sum_approved_loans,sum_all_loan_amount,denial_rate,avg_approved_loan_size,loan_approvel_rate
Median_Income_Quintiles,,,,,,,
Q5,29.641713,33.779516,2.895185e+07,3.232705e+07,0.826069,847175.262781,0.877506
Q4,41.197819,47.708161,2.200353e+07,2.516942e+07,0.839388,495327.260140,0.863538
Q3,47.028729,55.957274,1.920789e+07,2.240500e+07,0.824266,404393.783319,0.840440
Q2,41.756264,51.124146,1.371401e+07,1.645155e+07,0.806251,372658.732059,0.816762
Q1,33.341357,41.715536,7.835157e+06,9.586236e+06,0.778195,240966.973410,0.799255
